In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import re
from data_science_toolkit.string_ops import remove_newlines, remove_excess_spaces, normalize_links, lower, custom_replace, html_to_unicode, unicode_to_html
from data_science_toolkit.utils import parallel_compute
import nltk
from unidecode import unidecode
import numpy as np
from collections import defaultdict
from dateutil import relativedelta
from data_science_toolkit.file_ops import write_pkl
from ast import literal_eval

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

# CHANGE THIS WHEN USING A DIFFERENT TWITTER ACCOUNT!!

In [3]:
input_file_name = 'elonmusk_twint.csv'
output_file_name = "elonmusk_twint_preprocessed.csv"

In [4]:
trumptweets = pd.read_csv("./tweets/{}".format(input_file_name))
# trumptweets = trumptweets.dropna()
# stocks = pd.read_csv('./stocks/spx_preprocessed.csv')
# stocks['Date']= pd.to_datetime(stocks['Date']) 

In [5]:
trumptweets.head(5)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date
0,1199240674484162560,1190264525838262272,1574756232000,2019-11-26,03:17:12,Eastern Standard Time,44196397,elonmusk,Elon Musk,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk'}, {'user_id': '16504257', 'username': 'Nnedi'}, {'user_id': '13298072', 'username': 'Tesla'}]",NaN
1,1199158300257865729,1198751258384818176,1574736592000,2019-11-25,21:49:52,Eastern Standard Time,44196397,elonmusk,Elon Musk,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk'}, {'user_id': '258602317', 'username': 'kimpaquette'}, {'user_id': '19725644', 'username': 'neiltyson'}]",NaN
2,1199157485082312704,1198751258384818176,1574736398000,2019-11-25,21:46:38,Eastern Standard Time,44196397,elonmusk,Elon Musk,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk'}, {'user_id': '19725644', 'username': 'neiltyson'}]",NaN
3,1199093936070905856,1198751258384818176,1574721247000,2019-11-25,17:34:07,Eastern Standard Time,44196397,elonmusk,Elon Musk,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk'}, {'user_id': '19725644', 'username': 'neiltyson'}]",NaN
4,1199090292177006592,1199090292177006592,1574720378000,2019-11-25,17:19:38,Eastern Standard Time,44196397,elonmusk,Elon Musk,NaN,...,https://twitter.com/sundeep/status/1198988235252199424,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk'}]",NaN


In [6]:
trumptweets['tweet'].isna().sum()

0

In [7]:
trumptweets = trumptweets.dropna(axis=1)
# trumptweets = trumptweets.countna(subset=['tweet'])
trumptweets.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'tweet', 'mentions', 'urls', 'photos',
       'replies_count', 'retweets_count', 'likes_count', 'hashtags',
       'cashtags', 'link', 'retweet', 'video', 'reply_to'],
      dtype='object')

In [8]:
trumptweets.head(3)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,tweet,...,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,video,reply_to
0,1199240674484162560,1190264525838262272,1574756232000,2019-11-26,03:17:12,Eastern Standard Time,44196397,elonmusk,Elon Musk,We’d be happy to help,...,[],95,68,1626,[],[],https://twitter.com/elonmusk/status/1199240674484162560,False,0,"[{'user_id': '44196397', 'username': 'elonmusk'}, {'user_id': '16504257', 'username': 'Nnedi'}, {'user_id': '13298072', 'username': 'Tesla'}]"
1,1199158300257865729,1198751258384818176,1574736592000,2019-11-25,21:49:52,Eastern Standard Time,44196397,elonmusk,Elon Musk,"Sure, will aim to do this next week",...,[],240,237,5946,[],[],https://twitter.com/elonmusk/status/1199158300257865729,False,0,"[{'user_id': '44196397', 'username': 'elonmusk'}, {'user_id': '258602317', 'username': 'kimpaquette'}, {'user_id': '19725644', 'username': 'neiltyson'}]"
2,1199157485082312704,1198751258384818176,1574736398000,2019-11-25,21:46:38,Eastern Standard Time,44196397,elonmusk,Elon Musk,"Agreed, this will be exciting to see!",...,[],382,450,17301,[],[],https://twitter.com/elonmusk/status/1199157485082312704,False,0,"[{'user_id': '44196397', 'username': 'elonmusk'}, {'user_id': '19725644', 'username': 'neiltyson'}]"


In [9]:
trumptweets = trumptweets.drop(["id", 
                                "conversation_id", 
                                "created_at", 
                                "timezone", 
                                "user_id", 
                                "username", 
                                "name", 
                                "replies_count",
                                "likes_count",
                                "reply_to",
                                "cashtags",
                                "mentions",
                                "link"], axis=1)

In [10]:
trumptweets = trumptweets.rename(columns={"retweets_count": "retweet_count", 
                                          "tweet": "text", 
                                          "video": "num_videos"})

In [11]:
trumptweets.head(2)

,date,time,text,urls,photos,retweet_count,hashtags,retweet,num_videos
0,2019-11-26,03:17:12,We’d be happy to help,[],[],68,[],False,0
1,2019-11-25,21:49:52,"Sure, will aim to do this next week",[],[],237,[],False,0


In [12]:
trumptweets['created_at'] = trumptweets.date + " " + trumptweets.time

In [13]:
trumptweets.head(2)

,date,time,text,urls,photos,retweet_count,hashtags,retweet,num_videos,created_at
0,2019-11-26,03:17:12,We’d be happy to help,[],[],68,[],False,0,2019-11-26 03:17:12
1,2019-11-25,21:49:52,"Sure, will aim to do this next week",[],[],237,[],False,0,2019-11-25 21:49:52


In [14]:
trumptweets['created_at']= pd.to_datetime(trumptweets['created_at'], format="%Y-%m-%d %H:%M:%S") 

In [15]:
trumptweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8126 entries, 0 to 8125
Data columns (total 10 columns):
date             8126 non-null object
time             8126 non-null object
text             8126 non-null object
urls             8126 non-null object
photos           8126 non-null object
retweet_count    8126 non-null int64
hashtags         8126 non-null object
retweet          8126 non-null bool
num_videos       8126 non-null int64
created_at       8126 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](1), int64(2), object(6)
memory usage: 579.4+ KB


### Only keep original tweets

In [16]:
# trumptweets['retweet'] = (trumptweets['is_retweet'] == 'true').astype(bool)
trumptweets = trumptweets[trumptweets['retweet'] == False]

In [17]:
trumptweets = trumptweets.sort_values(by="created_at")
trumptweets = trumptweets.reset_index(drop=True)
# stocks = stocks.sort_values(by="Date")
# stocks = stocks.reset_index(drop=True)

In [18]:
# boi = trumptweets.iloc[3500].text
# boi

In [19]:
import demoji
def replace_emojis_with_desc(text):
    emoji_map = demoji.findall(text)
    for key in emoji_map.keys():
        emoji_map[key] = re.sub(r'\W+', '', emoji_map[key].lower()).replace(" ", "")
    for k, v in emoji_map.items():
        text = text.replace(k, v)
    return text
# boi

## Easy Text preprocessing
- Convert to lower case
- Convert links to `msciurl`
- Remove 'RT'
- Turn @mentions into `twitmention` (nltk splits the @ and the name)
- Turn #hashtags into `twithashtag` (nltk splits the # and the content)
- Tokenize sentence (split punctuation into its own tokens)
- Replace unicode characters with their ascii equivalents (makes things easier to analyze)
- Remove excess spaces/newlines

In [20]:
def preprocess_tweet(tweet):
    tweet = normalize_links(remove_newlines(html_to_unicode(tweet)), 'msciurl')
    tweet = re.sub(r"pic\.twitter\.com\S+", 'twitpicture', tweet)
    tweet = replace_emojis_with_desc(tweet)
    # NLTK by default turns mentions into @ <tag>.  Makes for easier analysis this stops the mentions from being separated, and allows them to be used in the clustering and stuff later...
    tweet = custom_replace(tweet,
                          [re.compile('RT'), re.compile(r"@(?=\S)"), re.compile(r"#(?=\S)")],
                          [' ', 'twitmention', 'twithashtag'])
    tweet = ' '.join([word for sent in nltk.sent_tokenize(tweet) for word in nltk.word_tokenize(sent)])
    return unidecode(lower(remove_excess_spaces(tweet)))

In [21]:
tweets = trumptweets["text"].tolist()
processed_tweets = parallel_compute(tweets, preprocess_tweet)

100%|██████████| 8126/8126 [00:00<00:00, 8392.78it/s]


In [22]:
trumptweets['preprocessed_text'] = processed_tweets

## Add Other Columns for Analysis

### Hour of day tweet was created

In [23]:
trumptweets["created_hour"] = trumptweets["created_at"].apply(lambda x: x.hour)

### Day of week tweet was created

In [24]:
trumptweets['dow'] = trumptweets['created_at'].dt.dayofweek

### Number of links in tweet

In [25]:
trumptweets['num_links'] = trumptweets.preprocessed_text.apply(lambda tweet: sum([1 if 'msciurl' in word else 0 for word in tweet.split("  ")]))

### Number of words in tweet (ignoring mentions, hashtags, and links)

In [26]:
trumptweets['num_words'] = trumptweets.preprocessed_text.apply(lambda tweet: sum([1 if len(word) > 1 and 'msciurl' not in word and 'twitmention' not in word and 'twithashtag' not in word else 0 for word in tweet.split(" ")]))

### Count Photos

In [27]:
trumptweets['num_photos'] = trumptweets.photos.apply(lambda x: len(literal_eval(str(x))))

## There is one tweets that is messed up... remove it

In [28]:
# trumptweets = trumptweets[trumptweets.num_words < 55]

In [29]:
trumptweets.head(5)

,date,time,text,urls,photos,retweet_count,hashtags,retweet,num_videos,created_at,preprocessed_text,created_hour,dow,num_links,num_words,num_photos
0,2010-06-04,14:31:57,"Please ignore prior tweets, as that was someone pretending to be me :) This is actually me.",[],[],122,[],False,0,2010-06-04 14:31:57,"please ignore prior tweets , as that was someone pretending to be me : ) this is actually me .",14,4,0,16,0
1,2011-12-01,04:55:11,"Went to Iceland on Sat to ride bumper cars on ice! No, not the country, Vlad's rink in Van Nuys. Awesome family fun :) pic.twitter.com/rBQXJ9IT",[],['https://pbs.twimg.com/media/Afkf5DMCAAEJGFr.jpg'],17,[],False,0,2011-12-01 04:55:11,"went to iceland on sat to ride bumper cars on ice ! no , not the country , vlad 's rink in van nuys . awesome family fun : ) twitpicture",4,3,0,25,1
2,2011-12-01,05:29:04,I made the volume on the Model S http://ow.ly/i/mtD7 go to 11. Now I just need to work in a miniature Stonehenge...,['http://ow.ly/i/mtD7'],[],9,[],False,0,2011-12-01 05:29:04,i made the volume on the model s msciurl go to 11 . now i just need to work in a miniature stonehenge ...,5,3,1,18,0
3,2011-12-03,03:20:28,"Great Voltaire quote, arguably better than Twain. Hearing news of his own death, Voltaire replied the reports were true, only premature.",[],[],25,[],False,0,2011-12-03 03:20:28,"great voltaire quote , arguably better than twain . hearing news of his own death , voltaire replied the reports were true , only premature .",3,5,0,21,0
4,2011-12-03,03:22:07,That was a total non sequitur btw,[],[],4,[],False,0,2011-12-03 03:22:07,that was a total non sequitur btw,3,5,0,6,0


# MORE OR LESS RETWEETS THAN AVG!!

In [30]:
min_year = min([x.year for x in trumptweets['created_at'].tolist()])
max_year = max([x.year for x in trumptweets['created_at'].tolist()])+1

In [31]:
curr_dt = datetime(min_year, 1, 1)
dt_avg = defaultdict(lambda: defaultdict(int))
while curr_dt < datetime(max_year, 1, 1):
    dt_avg[curr_dt.year][curr_dt.month] = np.mean(trumptweets[trumptweets['created_at'].between(curr_dt, curr_dt+relativedelta.relativedelta(months=1))]['retweet_count'].tolist())  
    curr_dt += relativedelta.relativedelta(months=1)

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Save averages to pkl to read in EDA.ipynb

In [32]:
write_pkl("dt_avg_{}.pkl".format(output_file_name), dict(dt_avg))

In [33]:
greater_than_monthly_avg = list(map(int, [count > dt_avg[curr_date.year][curr_date.month] 
                                          for count, curr_date in zip(trumptweets['retweet_count'].tolist(), trumptweets['created_at'].tolist())]))
trumptweets['above_monthly_avg'] = greater_than_monthly_avg

## Percent Caps

In [34]:
orig = trumptweets['text'].tolist()
percent_caps = []
for tweet in orig:
    words = tweet.split(" ")
    num_allcaps = sum([1 for x in words if x.upper() == x and re.search('[a-zA-Z]', x)])
    percent_caps.append(int(num_allcaps/len(words)*10)/10 if len(words) else 0)

In [35]:
trumptweets['percent_caps'] = percent_caps
print(set(percent_caps))

{0.0, 1.0, 0.1, 0.3, 0.2, 0.5, 0.4, 0.6}


## Num Hashtags

In [36]:
preprocessed = trumptweets['preprocessed_text'].tolist()
num_hashtags = []
for tweet in preprocessed:
    words = tweet.split(" ")
    num_hashtag = sum([1 for x in words if 'twithashtag' in x and re.search('[a-zA-Z]', x)])
    num_hashtags.append(num_hashtag if num_hashtag else 0)

In [37]:
trumptweets['num_hashtags'] = num_hashtags
print(set(num_hashtags))

{0, 1, 2}


## Num Mentions

In [38]:
num_mentions = []
for tweet in preprocessed:
    words = tweet.split(" ")
    num_mention = sum([1 for x in words if 'twitmention' in x and re.search('[a-zA-Z]', x)])
    num_mentions.append(num_mention if num_mention else 0)

In [39]:
trumptweets['num_mentions'] = num_mentions
print(set(num_mentions))

{0, 1, 2, 3, 4}


In [40]:
trumptweets = trumptweets.drop(["date", "time", "urls", "photos", "hashtags"], axis=1)

### Final Dataframe...

In [42]:
trumptweets.head(5)

,text,retweet_count,retweet,num_videos,created_at,preprocessed_text,created_hour,dow,num_links,num_words,num_photos,above_monthly_avg,percent_caps,num_hashtags,num_mentions
0,"Please ignore prior tweets, as that was someone pretending to be me :) This is actually me.",122,False,0,2010-06-04 14:31:57,"please ignore prior tweets , as that was someone pretending to be me : ) this is actually me .",14,4,0,16,0,0,0.0,0,0
1,"Went to Iceland on Sat to ride bumper cars on ice! No, not the country, Vlad's rink in Van Nuys. Awesome family fun :) pic.twitter.com/rBQXJ9IT",17,False,0,2011-12-01 04:55:11,"went to iceland on sat to ride bumper cars on ice ! no , not the country , vlad 's rink in van nuys . awesome family fun : ) twitpicture",4,3,0,25,1,0,0.0,0,0
2,I made the volume on the Model S http://ow.ly/i/mtD7 go to 11. Now I just need to work in a miniature Stonehenge...,9,False,0,2011-12-01 05:29:04,i made the volume on the model s msciurl go to 11 . now i just need to work in a miniature stonehenge ...,5,3,1,18,0,0,0.1,0,0
3,"Great Voltaire quote, arguably better than Twain. Hearing news of his own death, Voltaire replied the reports were true, only premature.",25,False,0,2011-12-03 03:20:28,"great voltaire quote , arguably better than twain . hearing news of his own death , voltaire replied the reports were true , only premature .",3,5,0,21,0,0,0.0,0,0
4,That was a total non sequitur btw,4,False,0,2011-12-03 03:22:07,that was a total non sequitur btw,3,5,0,6,0,0,0.0,0,0


### Save

In [43]:
trumptweets.to_csv("./stocks/{}".format(output_file_name), index=False)